In [1]:
#==============================================================================================
# 기존 bert vocab에 새로운 도메인 vocab만들어서 추가하기
#
# =>기존 pre-trained bert vocab에는 전문단어 domain이 없다(예:문서중앙화, COVID 등)  
# 따라서 전문 domain을 기존 bert vocab에 추가해야 한다.
# => 따라서 여기서는 mecab(한국어 형태로 분석기)를 이용하여, 추가 vocab을 만들고, 
# bert wordpiecetokenzer에 추가하는 방법에 대해 설명한다
# 
# 참고 자료 
# https://github.com/piegu/language-models/blob/master/nlp_how_to_add_a_domain_specific_vocabulary_new_tokens_to_a_subword_tokenizer_already_trained_like_BERT_WordPiece.ipynb
# https://medium.com/@pierre_guillou/nlp-how-to-add-a-domain-specific-vocabulary-new-tokens-to-a-subword-tokenizer-already-trained-33ab15613a41
# https://wikidocs.net/64517
#
# [과정]
# 1) 도메인 말뭉치(예:kowiki.txt)에서 mecab을 이용하여 형태소 분석하여 단어들을 추출함.
#   => mecab으로 형태소 혹은 명사만 분할하면서, subword 앞에는 '##' prefix 추가함(**ertwordpiece subword와 동일하게)
# 2) NLTK 빈도수 계산하는 FreqDist()를 이용하여, 단어들이 빈도수 계산
# 3) 상위 빈도수(예:30000)를 가지는 단어들만 add_vocab.txt 로 만듬
# 4) 기존 bert vocab.txt 파일에 직접, add_vocab.txt 토큰들 추가(*이때 중복 제거함)
# 5) 추가한 vocab을 가지고, tokenizer 생성하고, special 토큰 추가 후, 저장
#
# **원래 참고자료에는 tokenizer.add_tokens() 으로 추가하면, added_tokens.json 파일에 추가되는데, 
# 이때 BertTokenizer.from_pretrained() 함수로 호출할때, 호출이 안됨(**이유 모름:엄청 느려지는것 같음)
# => 따라서 직접 vocab.txt에 추가하는 방법을 사용함
#==============================================================================================
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import BertTokenizer

In [2]:
# 필요에 따라 load_dataset 에서 corpus 불러와서 data 폴더에 cache 파일로 저장.
# 이후 저장된 캐쉬경로에 파일을 이름변경해서 사용하면 됨.

#from datasets import load_dataset
#dataset = load_dataset('bongsoo/moco-corpus', cache_dir='./data')

Using custom data configuration bongsoo--moco-corpus-ef2b5bdf9d2fec8d


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/351M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to ./data/text/bongsoo--moco-corpus-ef2b5bdf9d2fec8d/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
#dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3291463
    })
})

In [23]:
#dataset['train']['text'][100]

'These are the result of inserts into index blocks when multipleinstances share a sequence generator for primary key values.'

In [3]:
# wiki_20190620.txt 말뭉치 불러옴.
#corpus = '../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt'
#corpus = '../korpora/kowiki_20190620/wiki_20190620.txt'
corpus = './data/moco-corpus.txt'
with open(corpus, 'r', encoding='utf-8') as f:
    data = f.read().split(' ') # 공백으로 구분해서 단어들을 추출함

print(data[:3])
print(len(data))

['Refer', 'to', 'the']
42843613


In [4]:
# mecab 형태소 분석기를 이용하여, 읽어온 말뭉치를 단어,조사등으로 분리함
# => 불용어는 제거함
# => mecab으로 형태소 분할하면서, subword 앞에는 prefix '##' 추가함

# Mecab 선언
mecab = Mecab()

# 불용어 정의
#stopwords=['이','가','께서','에서','이다','의','을','를','에','에게','께','와','에서','라고', '과','은', '는', '부터','.',',', '_']
stopwords=['.',',', '_', '..','"',';','<', '>', '(', ')', '[', ']']
# Ture = nouns(명사)만 추출, False=형태소 추출
nouns = False

In [5]:
'''
total_morph=[]
for sentence in tqdm(data):
    temp = mecab.morphs(sentence)
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    total_morph.append(temp)
'''
# mecab으로 형태소 혹은 명사만 분할할때, subword 앞에는 prefix '##' 추가함
total_words=[]

# 명사만 추출하는 경우
if nouns == True:
    for words in tqdm(data):
        count=0

        for word in mecab.nouns(words):
            if not word in stopwords:
                tmp = word

                if count > 0:
                    tmp = "##" + tmp  ## subword 앞에는 prefix '##' 추가함
                    total_words.append(tmp)  
                else:
                    total_words.append(tmp)  
                    count += 1
# 형태소도 포함하여 추출하는 경우
else:
    
    for words in tqdm(data):
        count=0

        for word in mecab.morphs(words):
            if not word in stopwords:
                tmp = word

                if count > 0:
                    tmp = "##" + tmp  ## subword 앞에는 prefix '##' 추가함
                    total_words.append(tmp)  
                else:
                    total_words.append(tmp)  
                    count += 1
                
print(total_words[:20])
print(f'총 단어 수: {len(total_words)}')

  0%|          | 0/42843613 [00:00<?, ?it/s]

['Refer', 'to', 'the', 'V', '##$', '##SYSTEM', '##EVENT', 'view', 'for', 'time', 'waited', 'and', 'average', 'waits', 'for', 'thefollowing', 'actions', '##:', '##To', 'estimate']
총 단어 수: 60529980


In [6]:
# FreqDist를 이용하여 빈도수 계산(*오래걸림)
vocab = FreqDist(np.hstack(total_words))

print('단어 집합의 크기 : {}'.format(len(vocab)))

# 최대 빈도수 가장높은 1500개만 뽑아봄
print(vocab.most_common(1500))

# 특정 단어의 빈도수를 뽑아봄
#print(vocab['미국'])


단어 집합의 크기 : 446597
[('the', 2184909), ('to', 832465), ('##하', 710062), ('##는', 692741), ('##을', 691642), ('a', 663946), ('of', 663657), ('is', 585547), ('##를', 583267), ('##에', 574573), ('and', 537129), ('in', 449548), ('##-', 397030), ('##의', 372288), ('##:', 359164), ('for', 350187), ('##이', 342137), ('that', 320852), ('##The', 316292), ('있', 280132), ('##은', 273927), ('you', 264463), ('##/', 241291), ('on', 236966), ('##한', 229810), ('##에서', 228057), ('are', 228015), ('be', 226832), ('##가', 223465), ('or', 222871), ('##으로', 221287), ('can', 199281), ('##여', 195543), ('및', 194967), ('수', 178942), ('this', 178025), ('##고', 172999), ('queue', 168580), ('an', 167647), ('with', 167047), ('##로', 166424), ('##할', 164394), ('not', 159734), ('##되', 158479), ('message', 154641), ('by', 154080), ('사용', 153926), ('as', 153561), ('##한다', 146497), (':', 132688), ('##This', 129445), ('##적', 128506), ('##된', 127689), ('##다', 121926), ('##과', 121765), ('경우', 120934), ('##합니다', 120493), ('it', 118365

In [7]:
# 상위 32000 개만 보존
vocab_size = 32000
vocab1 = vocab.most_common(vocab_size)

vocab_len = len(vocab1)
print('*단어 집합의 크기 : {}'.format(vocab_len))
print('*마지막 단어 정보 : {}'.format(vocab1[vocab_len-1]))

*단어 집합의 크기 : 32000
*마지막 단어 정보 : ('HAProxy', 51)


In [ ]:

'''
import seaborn as sns
import pandas as pd
vocab2 = vocab.most_common(10)

all_fdist = pd.Series(dict(vocab2))
fig, ax = plt.subplots(figsize=(10,10))
## Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=all_fdist.index, y=all_fdist.values, ax=ax)
plt.xticks(rotation=30);
'''

In [8]:
# vocab을 list로 만듬
new_vocab = []
for index, word in tqdm(enumerate(vocab1)):
    new_vocab.append(word[0])  # fword[0] 하면 단어만 추출
    
# 리스트 출력해봄
print(new_vocab[50:70])
#print(vocab['음악당'])


0it [00:00, ?it/s]

['##This', '##적', '##된', '##다', '##과', '경우', '##합니다', 'it', 'from', '##습니다', 'data', '##If', 'use', '데이터', '시스템', 'MQ', 'name', '##와', '##”', 'WebSphere']


In [9]:
# new_vocab을 파일로 저장함
new_vocab_out = './moco-vocab/moco-corpus-32000-vocab.txt'
with open(new_vocab_out, 'w', encoding='utf-8') as f:
    for word in tqdm(new_vocab):
        f.write(word+'\n')

  0%|          | 0/32000 [00:00<?, ?it/s]

In [10]:
# new_vocab 파일을 불러옴.
#new_vocab_out = 'new_vocab.txt'

# 2차 불용어 정의
#stopwords=['##다', '##하', '있', '##고', '##로', '##한', '##적', '##되', '##었', '##_']
stopwords=['##_', '〈', '..']

new_vocab = []
with open(new_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

for word in tqdm(data):
    if not word in stopwords:  # 2차 불용어는 제외
        new_vocab.append(word)
        
print(new_vocab[0:10])

  0%|          | 0/32001 [00:00<?, ?it/s]

['the', 'to', '##하', '##는', '##을', 'a', 'of', 'is', '##를', '##에']


In [12]:
from transformers import AutoTokenizer
# bongsoo/mdistilbertV1.1 모델 불러옴.
vocab_path="bongsoo/mdistilbertV1.1"

tokenizer = AutoTokenizer.from_pretrained(vocab_path, do_lower_case=False)
# fast 토크너나이즈인지 확인
print(f'{vocab_path} is_fast:{tokenizer.is_fast}')

bongsoo/mdistilbertV1.1 is_fast:True


In [14]:
# tokenizer 저장
SAVE_VOCAB_PATH = './moco-vocab/mdistilbertV1.1'
tokenizer.save_pretrained(SAVE_VOCAB_PATH)

('./moco-vocab/mdistilbertV1.1/tokenizer_config.json',
 './moco-vocab/mdistilbertV1.1/special_tokens_map.json',
 './moco-vocab/mdistilbertV1.1/vocab.txt',
 './moco-vocab/mdistilbertV1.1/added_tokens.json',
 './moco-vocab/mdistilbertV1.1/tokenizer.json')

In [15]:
# 원래 vocab 리스트로 읽어옴
org_vocab_out = './moco-vocab/mdistilbertV1.1/vocab.txt'
org_vocab = []

with open(org_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
    
for vocab in tqdm(data):
     org_vocab.append(vocab)

print(len(org_vocab))
print(org_vocab[1111:1115])

  0%|          | 0/146444 [00:00<?, ?it/s]

146444
['વ', 'શ', 'ષ', 'સ']


In [16]:
# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거
#temp = list(set(new_vocab) - set(org_vocab)) #순서 보존이 안됨
    
#또는

s = set(org_vocab)
temp = [x for x in new_vocab if x not in s] #순서 보존됨    

vocablist = org_vocab + temp  #원본 리스트 + 중복제가한 신규리스트 2개의 리스트를 합침

print(len(vocablist))
print(vocablist[1111:1115])

164315
['વ', 'શ', 'ષ', 'સ']


In [18]:
# 리스트를 파일로 저장
outfilepath = './moco-vocab/mdistilbertV1.2/vocab.txt'
with open(outfilepath, 'w') as f:
    f.write('\n'.join(vocablist))
    
print('vocab 파일"{}" 추가 성공!. 출력 vocab 파일 : "{}"'.format(new_vocab_out, outfilepath))

vocab 파일"./moco-vocab/moco-corpus-32000-vocab.txt" 추가 성공!. 출력 vocab 파일 : "./moco-vocab/mdistilbertV1.2/vocab.txt"


In [ ]:
## **** 저장된 파일을 열어서, 중간(mbert는 119548라인)에 공백이 없는지 반드시 확인.

## **** 반드시 또 추가할 단어들(예:문서중앙화, 보안파일서버, 엠파워, 모코엠시스, MOCOMSYS, MPOWER, EZis-C 등) 파일 맨 끝에 추가 *단 기존에 추가되어있는지 확인 필요

In [19]:
# BertTokenizer 로 새롭게 저장된 vocalfile 불러옴
tokenizer = BertTokenizer(vocab_file=outfilepath, 
                          strip_accents=False, 
                          do_lower_case=False)

In [20]:
add_special_token = False

if add_special_token:
    # 옵션 : 저장된 add_new_vocab.txt를 불러와서 special 토큰 추가함
    special_tokens=['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]',
                    '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]',]

    special_tokens_dict = {'additional_special_tokens': special_tokens}
    tokenizer.add_special_tokens(special_tokens_dict)

    # special token 체크
    print(tokenizer.all_special_tokens)

In [21]:
# special token 추가한 special vocab을 저장함
import os
OUT_PATH = './moco-vocab/mdistilbertV1.2/'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)

('./moco-vocab/mdistilbertV1.2/tokenizer_config.json',
 './moco-vocab/mdistilbertV1.2/special_tokens_map.json',
 './moco-vocab/mdistilbertV1.2/vocab.txt',
 './moco-vocab/mdistilbertV1.2/added_tokens.json')

In [ ]:
'''
#=============================================================================================
# add_tokens으로 추가하면, BertTokenizer.from_pretrained() 함수로 호출할때, 호출이 안됨(**이유 모름)
# => 따라서 직접 vocab.txt에 추가하는 방법을 사용함
#=============================================================================================

# 기존 bert tokenizer 로딩
vocab_path = '../model/bert/bert-multilingual-cased/vocab'
tokenizer = BertTokenizer.from_pretrained(vocab_path)
print(f'*기존 vocab 수: {len(tokenizer)}')

# 기존 bert tokenizer에 신규 vocab 추가함 
new_token_num = tokenizer.add_tokens(new_vocab)
print(f'*신규 vocab 수: {len(tokenizer)}')
print(f'*추가된 vocab: {new_token_num}')

# 추가된 tokenizer 저장
# => 추가된 token들은 added_tokens.json에 저장된다.
import os
OUT_PATH = 'new_vocab_nouns'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)
'''